<a href="https://colab.research.google.com/github/emzu/probabilistic-urban-evolution/blob/main/Footprint_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Score accuracy compared to Ground Truth, Google Buildings dataset
---



Load Google Opean Buildings Footprints Data \\
[Access Data Download](https://colab.research.google.com/github/google-research/google-research/blob/master/building_detection/open_buildings_download_region_polygons.ipynb#scrollTo=qP6ADuzRdZTF) \\
[Dataset Reference](https://sites.research.google/open-buildings/#dataformat) \\
[Dataset Technical Report]() \\

Time-Frame  
  v1: inference carried out during April 2021 on imagery covering 19.4M km2 of Africa.

  v2: inference carried out during August 2022 on imagery covering 39.1M km2 of Africa, South and South-East Asia.

  v3: inference carried out during May 2023 on imagery covering 58M km2 of Africa, South and South-East Asia, Latin America and the Caribbean. See FAQ for comparison of versions.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Research/Morphology/Data/Building Footprints/Google Buildings Footprints/tmpv2fvxs5a", engine="c",
    usecols=['latitude', 'longitude', 'area_in_meters', 'confidence'])
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326).to_crs(32736)

### Load Validation Areas

In [ ]:
validation_gdf = gpd.read_file("/content/drive/MyDrive/Research/Morphology/Data/Validation Areas/validationAreas (1).shp").to_crs(32736)

In [ ]:
#Include Local CLimate Zones
data_lcz = pd.read_csv('/content/drive/MyDrive/Research/Morphology/Data/LCZ_dict.csv', header=None).set_index(0)
LCZ_dict = data_lcz.to_dict(orient='dict')
data_val = pd.read_csv('/content/drive/MyDrive/Research/Morphology/Data/val_areas_lcz.csv').set_index('id')
val_lcz_dict = data_val.to_dict(orient="dict")
val_lcz_dict=val_lcz_dict['lcz']
lcz_rec = []
for id in range(28):
  lcz_rec.append(LCZ_dict[1][val_lcz_dict[id]])

In [ ]:
id=7
model_conf = pixel_df[pixel_df.intersects(validation_gdf.loc[id,:].geometry)]
google_conf = gdf[gdf.intersects(validation_gdf.loc[id,:].geometry)]

filepath = "/content/drive/MyDrive/Building Footprints-Validation Areas/Shapefiles/"
code = "2022_0126"
year = "2022"
city = "Kigali"
filename = filepath+city+"/"+year+"/Label_"+code+"_20"+str(id)+".shp"
if os.path.isfile(filename):
  gt = gpd.read_file(filename).to_crs(32736)
  gt = gt[gt.geometry.type == 'Polygon'] #Only Keep Valid Polygon Geometry
  gt["value"]=1
#pixel_df = pixel_df.buffer(3)

In [ ]:
fig, ax = plt.subplots()
google_conf.plot(column = "confidence", ax=ax)
#model_conf.plot(column = "value", ax=ax)
gt.plot(ax=ax, color = "black")

### Calculate Cross Entropy Against Ground Truth